In [1]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

In [2]:
from julep import Julep
import os
import yaml 
import time
from dotenv import load_dotenv

load_dotenv()

client = Julep(
    api_key=os.getenv('JULEP_API_KEY'),
    environment=os.getenv('JULEP_ENVIRONMENT')
)

MAILGUN_PASSWORD = os.environ['MAILGUN_PASSWORD']

In [3]:
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Julep Email Assistant",
    about=(
        "You are an agent that handles emails for julep users."
    ),
    model="claude-3.5-sonnet",
)

In [4]:
task_def = yaml.safe_load(f"""
# testing email sending for now
name: Study Reminder Bot
description: Sends a reminder email every 15 minutes to help with study consistency. 

input_schema:
  type: object
  properties:
    body:
      type: string
    from_email:
      type: string
    subject:
      type: string
    to:
      type: string

tools:
- name: send_email
  type: integration
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: {MAILGUN_PASSWORD}
      port: 587
      user: postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org

main:
  - tool: send_email
    arguments:
      body: Hey!
      from: postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org
      subject: www
      to: akshat_s1@mt.iitr.ac.in
""")
  # - tool: send_email
  #   arguments:
  #     body: $ steps[0].input.body 
  #     from_email: $ steps[0].input.from 
  #     subject: $ steps[0].input.subject
  #     to:  $ steps[0].input.to

In [5]:
task = client.tasks.create_or_update(
    agent_id=AGENT_UUID,
    task_id=TASK_UUID,
    **task_def,
)

In [6]:
execution = client.executions.create(
    task_id=task.id,
    input={
        "from": "postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org", 
        "to": "akshat_s1@mt.iitr.ac.in", 
        "subject": "Time to Study!", 
        "body": "Hey! Just a gentle nudge it's time to hit the books.Stay focused and take short breaks every hour. You got this!,", 
        }
)

In [7]:
client.executions.get(execution.id)

Execution(id='0680153c-258a-7449-8000-e61778494e6d', created_at=datetime.datetime(2025, 4, 17, 19, 17, 22, 350802, tzinfo=datetime.timezone.utc), input={'to': 'akshat_s1@mt.iitr.ac.in', 'body': "Hey! Just a gentle nudge it's time to hit the books.Stay focused and take short breaks every hour. You got this!,", 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': 'Time to Study!'}, status='starting', task_id='d0dad54d-d158-43a5-840b-0fb7c7542962', updated_at=datetime.datetime(2025, 4, 17, 19, 17, 22, 624439, tzinfo=datetime.timezone.utc), error=None, metadata={}, output={'to': 'akshat_s1@mt.iitr.ac.in', 'body': "Hey! Just a gentle nudge it's time to hit the books.Stay focused and take short breaks every hour. You got this!,", 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': 'Time to Study!'}, transition_count=1)

In [8]:
# Wait for the execution to complete
while (result := client.executions.get(execution.id)).status not in ['succeeded', 'failed']:
    print(f"Execution status: {result.status}")
    time.sleep(1)

if result.status == "succeeded":
    print("Email sent successfully!")
    print(result.output)
else:
    print(f"Error: {result.error}")


Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
Execution status: starting
E

KeyboardInterrupt: 

In [ ]:
execution_transitions = client.executions.transitions.list(
    execution_id=execution.id).items

for transition in reversed(execution_transitions):
    print("Type: ", transition.type)
    print("output: ", transition.output)
    print("-" * 100)

Type:  init
output:  {'to': 'akshat_s1@mt.iitr.ac.in', 'body': "Hey! Just a gentle nudge it's time to hit the books.Stay focused and take short breaks every hour. You got this!,", 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': 'Time to Study!'}
----------------------------------------------------------------------------------------------------


In [10]:
# Create an agent
agent = client.agents.create(
    name="Chat Buddy",
    about="A friendly and helpful chatbot",
    instructions=[
        "Be friendly and engaging.",
        "Be helpful and provide useful information.",
        "Be concise and to the point.",
        "Do not format your responses. Keep them as plain text.",
    ],
    model="gpt-4o-mini",
)

# Create a session
session = client.sessions.create(
    agent=agent.id,
    situation="User wants to have a casual chat about hobbies.",
)

# Chat with the agent
response = client.sessions.chat(
    session_id=session.id,
    messages=[
        {
            "role": "user",
            "content": "Hi there! What are some fun hobbies to try out?"
        }
    ]
)

In [11]:
print("Agent's Response:")
print(response.choices[0].message.content)

Agent's Response:
Hey! There are so many fun hobbies to explore! Here are a few you might enjoy: 

1. **Photography** - Capture moments and improve your skills with a camera or even your smartphone.
2. **Gardening** - Great for relaxation and enjoying nature, plus you can grow your own herbs or flowers.
3. **Cooking or Baking** - Experimenting with new recipes can be really satisfying.
4. **Drawing or Painting** - A creative outlet that lets you express yourself and unwind.
5. **Hiking** - Perfect for getting outdoors and staying active while enjoying beautiful views.
6. **Learning an Instrument** - It’s never too late to pick up a guitar, piano, or any instrument that interests you!
7. **Reading** - Dive into different worlds through books; there’s always something new to discover.
8. **Crafting** - Try your hand at DIY projects, knitting, or scrapbooking.

Which of these sounds interesting to you?
